In [14]:
import tkinter as tk
from tkinter.ttk import *
import numpy as np
import time


In [129]:
class Cell:
    def __init__(self,grid,row,col,state):
        self.grid = grid
        self.row = row
        self.col = col
        self.state = state
        self.neighbors = []
    def appear(self):
        self.state = 1
    def dead(self):
        self.state = 0
    def update_state(self):
        if self.state == 1:
            if (self.grid.counts[self.row][self.col]>3) or (self.grid.counts[self.row][self.col]<2):
                self.dead()
        else:
            if (self.grid.counts[self.row][self.col]==3):
                self.appear()

                
class Grid:
    def __init__(self,n_rows,n_cols):
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.cells = np.array([[Cell(self,row,col,0) for col in range(0,n_cols+2)] for row in range(0,n_rows+2)])
        # add 2 cols and rows for count_align_neighbors
        
        self.get_neighbors()
        self.count_align_neighbors()
        self.counts = self.count_align_neighbors()
    
    def get_neighbors(self):
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                neighbors = []
                for i in range(-1,2):
                    for j in range(-1,2):
                        neighbors.append(self.cells[row+i][col+j])
                del neighbors[4]                 #minus itself
                self.cells[row][col].neighbors = neighbors
                
    def count_align_neighbors(self):
        self.counts = [[0 for col in range(0,n_cols+2)] for row in range(0,n_rows+2)]
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                neighbors_state = 0
                
                for neighbor in self.cells[row][col].neighbors:
                    neighbors_state += neighbor.state
                    
                self.counts[row][col] = neighbors_state 
        return self.counts
    def update_grid_state(self):
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                self.cells[row][col].update_state()
class Click:  
    def Update_frame(grid,n_rows,n_cols,cell_buttons):
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                if grid.cells[row][col].state==1:
                    cell_buttons[row-1][col-1].config(bg='yellow',text='',height=0,width=0)
                else:
                    cell_buttons[row-1][col-1].config(bg='gray',text='',height=0,width=0)
    def Update_state_after_move(grid,state_arr):   #after move the frame
        print('Before')
        for i in range(n_rows+2):
            for j in range(n_cols+2):
                print(grid.cells[i][j].state,end='\t')
            print('\n')
        
        print('state_arr')
        print(state_arr)
        print('\n------------------------')
        for i in range(n_rows+2):
            for j in range(n_cols+2):
                grid.cells[i][j].state = state_arr[i][j]
                
        print('After')
        for i in range(n_rows+2):
            for j in range(n_cols+2):
                print(grid.cells[i][j].state,end='\t')
            print('\n')
    def Choose_cell_button(grid,cell_buttons,row, col):
        button = cell_buttons[row][col]
        button.config(bg="#14f500", text="OK",height=0,width=0)
        
        #Appear
        grid.cells[row+1][col+1].appear()
        choose_cell_label = tk.Label(root,text=f"Bạn đã chọn ô hàng {row+1}, cột {col+1}",bg='skyblue')
        choose_cell_label.place(x=50,y=380)
    
        next_grid_button = tk.Button(root,text='Start',command=lambda:Click.Start(grid,cell_buttons))
        next_grid_button.place(x=300,y=380)

    def Start(grid,cell_buttons):
        Click.Update_frame(grid,n_rows,n_cols,cell_buttons)
        next_grid_button = tk.Button(root,text='Next',command=lambda:Click.Next(grid,cell_buttons))
        next_grid_button.place(x=300,y=380)
        
        next_100_grid_button = tk.Button(root,text='Next_100',command=lambda:Click.Next_100(grid,cell_buttons))
        next_100_grid_button.place(x=350,y=380)
        
    def Up_frame(grid,cell_buttons):
        num_move = n_rows//2
        new_rows = [[Cell(grid,row,col,0) for col in range(0,n_cols+2)] for row in range(num_move)]
        
        grid.cells = np.vstack((grid.cells, new_rows))
        grid.cells = grid.cells[num_move:]
        
    def Down_frame(grid,cell_buttons):
        num_move = n_rows//2
        state_arr = [[grid.cells[i][j].state for j in range(0,n_cols+2)] for i in range(0,n_rows+2)]
        new_rows = [[0 for col in range(0,n_cols+2)] for row in range(num_move)]
        added_rows_grid = np.vstack((state_arr,new_rows))
        state_arr = added_rows_grid[num_move:]
        Click.Update_state_after_move(grid,state_arr)
        Click.Start(grid,cell_buttons)
    def Left_frame(grid,cell_buttons):
        print('Left_frame')
        
    def Right_frame(grid,cell_buttons):
        print('Right_frame')
    
    def warning_window(grid,type_warning,cell_buttons):
        message_box = tk.Toplevel(root)
        message_box.title("Cảnh báo") 
        warning_label = tk.Label(message_box, text="Ô sống đã vượt biên, bạn có muốn mở rộng biên không?")
        warning_label.pack()
        
        if type_warning =='up':
            update_button = tk.Button(message_box, text="Mở rộng lên", command=lambda:[Click.Up_frame(grid,cell_buttons),message_box.destroy()])
        if type_warning =='down':
            update_button = tk.Button(message_box, text="Mở rộng xuống", command=lambda:[Click.Down_frame(grid,cell_buttons),message_box.destroy()])
        if type_warning =='left':
            update_button = tk.Button(message_box, text="Mở rộng trái", command=lambda:[Click.Left_frame(grid,cell_buttons),message_box.destroy()])
        if type_warning =='right':
            update_button = tk.Button(message_box, text="Mở rộng phải", command=lambda:[Click.Right_frame(grid,cell_buttons),message_box.destroy()])
        
        update_button.pack()
        message_box.wait_window()
        
    def update_bounding(grid,cell_buttons):
        for i in range(1,n_cols+1):
            if grid.cells[1][i].state == 1:
                Click.warning_window(grid,'up',cell_buttons)
            if grid.cells[n_rows][i].state ==1:
                Click.warning_window(grid,'down',cell_buttons)
        for i in range(1,n_rows+1):
            if grid.cells[i][1].state == 1:
                Click.warning_window(grid,'left',cell_buttons)
            if grid.cells[i][n_cols].state == 1:
                Click.warning_window(grid,'right',cell_buttons)
    def Next(grid,cell_buttons):
        grid.count_align_neighbors()
        grid.update_grid_state()
        grid.count_align_neighbors()
        Click.Update_frame(grid,n_rows,n_cols,cell_buttons)
        Click.update_bounding(grid,cell_buttons) 
        
        next_grid_button = tk.Button(root,text='Next',command=lambda:Click.Next(grid,cell_buttons))
        next_grid_button.place(x=300,y=380)
        restart_button = tk.Button(root,text='Restart',command=lambda:Click.Restart(grid,cell_buttons))
        restart_button.place(x=450,y=380)
        
        

    def Next_100(grid,cell_buttons):
        for i in range(100):
            next_grid_button = tk.Button(root,text='Next',command=lambda:Click.Next(grid,cell_buttons))
            next_grid_button.invoke()
            root.update()
            Click.update_bounding(grid,cell_buttons)
            time.sleep(0.2)

        
    def Restart(grid,cell_buttons):
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                grid.cells[row][col].dead()
        Click.Update_frame(grid,n_rows,n_cols,cell_buttons)
        next_grid_button = tk.Button(root,text='Next',command=lambda:Click.Next(grid,cell_buttons))
        next_grid_button.place(x=300,y=380)
        
        
class create_click:
    def create_grid_button():
        grid = Grid(n_rows,n_cols)
        
        # reframe
        overlay = tk.Canvas(root, bg="gray", highlightthickness=0)
        overlay.place(relwidth=1, relheight=1)
        
        # margin
        for i in range(n_rows + 1):
            root.rowconfigure(i, weight=1)
        for j in range(n_cols + 1):
            root.columnconfigure(j, weight=1)
        
        # create button
        cell_buttons = [[None] * n_cols for _ in range(n_rows)]
        for i in range(n_rows):
            for j in range(n_cols):
                cell_button = tk.Button(root, text="{},{}".format(i+1, j+1),
                                   command=lambda row=i, col=j: Click.Choose_cell_button(grid,cell_buttons,row, col))
                cell_button.grid(row=i, column=j,sticky="nsew")
                cell_buttons[i][j] = cell_button
def get_num():
    try:
        global n_rows,n_cols
        n_rows = int(n_rows_entry.get())
        n_cols = int(n_cols_entry.get())
        if isinstance(n_rows,int) & isinstance(n_cols,int) & (n_rows>=5) & (n_cols>=5) & (n_rows<20) & (n_cols<20):
            get_num_checked_label = tk.Label(root,text="Số thỏa mãn,chơi thôi",fg='green')
            get_num_checked_label.place(x=150,y=130)
            
            start_btn = Button(root,text="Chơi",command=create_click.create_grid_button)
            start_btn.place(x=250,y=300)
            
        else:
            get_num_error_label = tk.Label(root,text="Nhập số không thỏa mãn\n Hãy nhập số nguyên, và lớn hơn 4 và không vượt quá 20 cho trải nghiệm chơi tốt nhất",
                                       fg='red')
            get_num_error_label.place(x=50,y=160)
    except:
        get_num_error_label = tk.Label(root,text="Nhập số không thỏa mãn\n Hãy nhập số nguyên, và lớn hơn 4 và không vượt quá 20 cho trải nghiệm chơi tốt nhất",
                                       fg='red')
        get_num_error_label.place(x=50,y=160)

In [130]:
root = tk.Tk()
root.geometry('600x400')
root.resizable(0,0)
root.title('Conway Game')
root.resizable(width=False, height=False)


# Label
n_rows_entry_label = tk.Label(root,text="Nhập số hàng:")
n_rows_entry_label.place(x=50,y=50)

n_cols_entry_label = tk.Label(root,text="Nhập số cột:")
n_cols_entry_label.place(x=50,y=100)

# Entry
n_rows_entry = tk.Entry(root)
n_rows_entry.place(x=150,y=50,width=20)

n_cols_entry = tk.Entry(root)
n_cols_entry.place(x=150,y=100,width=20)

# Button
start_button = tk.Button(root,text='Lấy số',command=get_num)
start_button.place(x=100,y=130)


root.mainloop()

Before
0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	1	0	0	0	0	0	0	

0	0	0	1	0	0	0	0	0	0	

0	0	0	1	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

state_arr
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

------------------------
After
0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	1	0	0	0	0	0	0	

0	0	0	1	0	0	0	0	0	0	

0	0	0	1	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

Before
0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

0	0	0	1	0	1	0	0	0	0	

0	0	0	0	1	1	0	0	0	0	

0	0	0	0	1	0	0	0	0	0	

0	0	0	0	0	0	0	0	0	0	

state_arr
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 

In [79]:
def print_cell(grid):
    for i in range(1,n_rows+1):
        for j in range(1,n_cols+1):
            print(grid.cells[i][j].state,end='\t')
        print('\n')
    print('----------------------------------------------------------')
    
def print_idx_cell(grid):
    for i in range(1,n_rows+1):
        for j in range(1,n_cols+1):
            print(f'[{i},{j}]=[{grid.cells[i][j].row},{grid.cells[i][j].col}]',end='\t')
        print('\n')
    print('----------------------------------------------------------')

In [50]:
new_rows = np.array([[1 for row in range(5+2)] for col in range(3)])

matrix = np.vstack((matrix,new_rows))
matrix = matrix[3:]
matrix

array([[0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])

In [118]:
matrix = [[0 for i in range(0,n_cols+2)] for j in range(0,n_rows+2)]
matrix

[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [121]:

state_arr = [[matrix[i][j] for i in range(0,n_cols+2)] for j in range(0,n_rows+2)]
new_rows = [[1 for col in range(0,n_cols+2)] for row in range(3)]
added_rows_grid = np.vstack((state_arr,new_rows))
state_arr = added_rows_grid[3:]

In [122]:
state_arr

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1]])